In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [5]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "qwiklabs-gcp-04-1c981773b2bb"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [3]:
model = GenerativeModel("gemini-1.5-flash")

In [4]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [5]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some names for a dried flower shop, playing on different themes:

**Nostalgic & Romantic:**

* **The Dried Petal**
* **Whispers of Wildflowers**
* **Forever Blooms**
* **The Vintage Bouquet**
* **Timeless Florals**

**Modern & Minimalist:**

* **Bloom & Wither**
* **Paper Petal**
* **The Dried Garden**
* **Botanical Muse**
* **Bloom & Preserve**

**Whimsical & Unique:**

* **The Flower Alchemist**
* **Dusty Blooms**
* **Sun-Kissed Blooms**
* **The Dried Flower Co.**
* **The Botanical Curio**

**Location Specific:**

* **[Your Town] Dried Flowers**
* **[Your Neighborhood] Bloomery**
* **[Your Street Name] Flower House**

**Tips for Choosing a Name:**

* **Keep it short and memorable.**
* **Consider your target audience.**
* **Make sure the name is available as a website domain and social media handle.**
* **Check if the name is already being used by another business.**
* **Say the name out loud to make sure it rolls off the tongue easily.**

Ultimately, the best name for your f

✅ Recommended. The prompt below is to the point and concise.

In [6]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

Here are some names for a flower shop that sells bouquets of dried flowers, playing on different themes:

**Classic & Elegant:**

* The Dried Bloom
* Everlasting Blooms
* Bloom & Wither
* The Dried Bouquet
* The Floral Archive
* Whispers of Bloom

**Modern & Minimalistic:**

* Bloom & Co.
* Dried.
* Everbloom
* Petal Paper
* Bloom & Shade

**Rustic & Charming:**

* The Dried Flower Barn
* The Wildflower Co.
* The Meadow Bouquet
* Gather & Bloom
* The Dried Garden
* The Flower Press

**Unique & Playful:**

* Bloom & Dust
* The Paper Petal
* The Immortal Bouquet
* Forever in Bloom
* The Botanical Archive
* The Pressed Petal

**Tips for Choosing a Name:**

* **Keep it short and memorable.**
* **Reflect your brand's aesthetic.**
* **Make sure it's easy to pronounce and spell.**
* **Check for availability online and in your area.**
* **Get feedback from friends and family.**

**Bonus tip:** Consider incorporating a geographical element or your personal touch in the name to make it even more

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [9]:
prompt = "Tell me how we can live in earth"

print(call_gemini(prompt))

It seems you're asking how we can **continue living on Earth**.  Here's a breakdown of what that means:

**1. Sustainable Living:**

* **Conserving Resources:** We need to be mindful of using water, energy, and natural resources wisely. This includes reducing waste, using renewable energy sources, and practicing responsible consumption.
* **Protecting Ecosystems:**  Maintaining healthy ecosystems like forests, oceans, and grasslands is crucial for clean air, water, and a stable climate. This involves combating pollution, deforestation, and habitat destruction. 
* **Managing Population Growth:**  Balancing population growth with Earth's carrying capacity is essential for sustainable living. This involves promoting family planning and addressing social and economic factors that influence population growth.

**2. Addressing Climate Change:**

* **Reducing Greenhouse Gas Emissions:**  We need to transition to cleaner energy sources, improve energy efficiency, and change our transportation 

✅ Recommended. The prompt below is specific and well-defined.

In [8]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## What Makes Earth Unique:

**1. Liquid Water:** Earth is the only known planet in our solar system with abundant liquid water on its surface. This is crucial for life as we know it, providing a medium for chemical reactions and transport, and acting as a temperature regulator. 

**2. Plate Tectonics:** Earth's crust is divided into plates that move and interact, causing earthquakes, volcanic eruptions, and the formation of mountains and ocean basins. This dynamic process helps regulate the planet's climate and recycles nutrients.

**3. Atmosphere:** Earth's atmosphere, primarily composed of nitrogen and oxygen, is a protective shield against harmful radiation from the Sun. It also traps heat, creating a hospitable climate and allowing for weather patterns. 

**4. Strong Magnetic Field:**  Earth's magnetic field acts as a shield against solar wind, protecting us from harmful charged particles. This shield is generated by the movement of molten iron in Earth's core.

**5. Diverse Life:

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [10]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Let's tackle these two questions separately:

**Boiling Water:**

There's no single "best" method for boiling water, as it depends on your needs and resources. Here are some common methods and their pros and cons:

* **Stovetop:**
    * **Pros:**  Versatile, widely available, usually quick.
    * **Cons:** Can be less efficient than other methods, requires constant monitoring.
* **Electric Kettle:**
    * **Pros:** Fast, energy-efficient, automatic shut-off for safety.
    * **Cons:** Can be bulky to store, only boils a limited amount of water.
* **Microwave:**
    * **Pros:** Fast for small amounts of water, convenient.
    * **Cons:**  Doesn't heat water evenly, potential for splattering.
* **Camping Stove:**
    * **Pros:** Portable, ideal for outdoor use.
    * **Cons:** Requires fuel, can be less efficient than other methods.

**The Sky's Blue Hue:**

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's the breakdown:

* **Sunlight:** Sunlight is made up

✅ Recommended. The prompts below asks one task a time.

In [11]:
prompt = "answer me in portuguese,What's the best method of boiling water?"

print(call_gemini(prompt))

O melhor método para ferver água depende do seu objetivo e dos recursos disponíveis. Mas, em geral, o método mais eficiente e seguro é:

**1. Ferver em uma panela:**

* **Escolha uma panela adequada:** Use uma panela com fundo grosso e tamanho adequado à quantidade de água que você precisa ferver.
* **Encha a panela com água:** Adicione a quantidade desejada de água, deixando um pouco de espaço para que a água não transborde durante o processo de fervura.
* **Aqueça a água:** Coloque a panela no fogão e ligue o fogo médio-alto.
* **Monitore a água:** Observe a água enquanto ela aquece. Quando começar a borbulhar e formar bolhas grandes e constantes, significa que a água ferveu.
* **Desligue o fogo:** Retire a panela do fogo e use a água fervida como desejar.

**Dicas:**

* **Use água fria:** A água fria ferve mais rápido do que água morna ou quente.
* **Não sobrecarregue a panela:** Deixar muito espaço livre na panela permite que a água ferva mais rapidamente e evita que transborde.
* 

In [13]:
prompt = "me responda em portugues, Why is the sky blue?"

print(call_gemini(prompt))

O céu é azul devido à dispersão de Rayleigh da luz solar pela atmosfera terrestre. 

A luz do sol é composta por todas as cores do arco-íris. Quando a luz solar atinge a atmosfera, ela interage com as moléculas de ar, principalmente nitrogênio e oxigênio. As moléculas de ar são muito menores que o comprimento de onda da luz visível, e é por isso que a dispersão de Rayleigh ocorre. 

A dispersão de Rayleigh faz com que a luz azul, que tem um comprimento de onda menor, seja espalhada em todas as direções muito mais do que outras cores, como o vermelho, que tem um comprimento de onda maior. Essa dispersão da luz azul faz com que o céu pareça azul. 

É por isso que o céu parece mais azul em dias claros e menos azul em dias nublados. Quando o céu está nublado, as nuvens refletem todas as cores da luz solar igualmente, o que faz com que o céu pareça branco.



### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [14]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To know what day it is today, please check a calendar or ask a device with internet access. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [15]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan is a city full of amazing sights! To give you the best recommendation, I need a little more information. What are you most interested in seeing? 

* **Art and History:**  The Duomo,  Sforza Castle,  Leonardo da Vinci's Last Supper, Pinacoteca di Brera
* **Fashion and Shopping:**  Via Montenapoleone, Galleria Vittorio Emanuele II, Corso Buenos Aires
* **Food and Culture:**  Navigli canals,  Mercato Centrale,  Brera district

Tell me what you're most interested in, and I can recommend the best places for you to visit! 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [16]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm designed to help travelers with planning their trips, and that includes finding restaurants and recommending dishes.  But I can't tell you what to eat for dinner tonight!  😊 

Do you have any questions about restaurants in Milan? Perhaps you're looking for a specific type of cuisine, or a restaurant with a certain atmosphere? 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [17]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

Here are some programming activities you can try, tailored to different skill levels and interests:

**Beginner:**

* **Build a text-based adventure game:** Use simple logic and conditional statements to create a story where the user makes choices that affect the outcome.  You can find tutorials online that break down the process. 
* **Create a simple calculator:** This project helps you practice basic arithmetic operations and input/output techniques.
* **Learn to code a basic website:** You can use HTML, CSS, and a bit of JavaScript to create a simple website with text, images, and links.

**Intermediate:**

* **Develop a simple game using a game library:** Use a library like Pygame (for Python) or Unity (for C#) to create a simple 2D game. This will teach you about game mechanics, graphics, and event handling.
* **Build a basic command-line tool:**  Learn to interact with the user through the command line, read files, and perform simple tasks.  
* **Contribute to an open-source proj

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [18]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
d) learn Java
"""

print(call_gemini(prompt))

It's great that you're thinking about learning programming! Here's a breakdown of the languages you listed, along with factors to consider when making your choice:

**a) Learn Python:**

* **Pros:**
    * **Beginner-friendly:** Python is known for its clean syntax and readability, making it a great starting point for beginners.
    * **Versatile:**  Used in a wide range of applications, from web development and data analysis to machine learning and scientific computing.
    * **Large Community & Resources:**  Strong support network, abundant online tutorials, and libraries. 
* **Cons:**
    * **Performance:**  Can be slower than compiled languages like C or Java for computationally intensive tasks.
* **Who it's good for:** Beginners, those interested in data science, web development, or general-purpose programming.

**b) Learn JavaScript:**

* **Pros:**
    * **Web Dominance:**  Essential for front-end web development, adding interactivity to websites. 
    * **Node.js for Back-end:** 

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [20]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I hate the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Negative** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [21]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [22]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 

Here's why:

* **"Something surprised me"**  implies a positive experience.
* **"It was actually original"** is a clear positive statement.
* **"It was not the same old recycled stuff"**  indicates a positive departure from the norm. 
* **"Watch it - you will not regret it"** strongly suggests the video is enjoyable. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.